## Imports

In [6]:
from pathlib import Path
import numpy as np
from PIL import Image

from pkg import models

repo_dir = Path().resolve().parent

## Model definition

In [2]:
model_names = ['microsoft/resnet-50', 'Visual-Attention-Network/van-base']

model_name = model_names[1]

model = models.GlassProbaPredictor(model_name)

## Data inference example

In [10]:
import urllib.request

data_dir = repo_dir / 'data'
data_dir.mkdir(exist_ok=True)

img_fp = data_dir / 'img.jpeg'

urllib.request.urlretrieve(
    'https://ae01.alicdn.com/kf/HTB14bHeBLuSBuNkHFqDq6xfhVXav/Ralferty-Transparent-Glasses-Trendy-Eyeglass-Woman-Frame-Female-Optic-Frames-Clear-Lens-Eyewear-Accessories-oculos-de.jpg', 
    img_fp
)

HTTPError: HTTP Error 403: Forbidden

In [12]:
im = Image.open(img_fp)
glass_prob = model.predict_proba(np.array(im))

print(f'Glasses prob: {glass_prob:.2f}')

Glasses prob: 0.17
